# Using the deafrica-waterbodies command line tool

In [4]:
# Uncomment the line below to install deafrica-waterbodies
!python -m pip install ../.

Defaulting to user installation because normal site-packages is not writeable
Processing /home/jovyan/dev/deafrica-waterbodies
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for deafrica-waterbodies: filename=deafrica_waterbodies-0.0.1-py3-none-any.whl size=43050 sha256=cda5bce16145ff9325190dc0482af19cd796fd50f18b7034511c381fd1eb134b
  Stored in directory: /home/jovyan/.cache/pip/wheels/d4/26/5e/6c4acf8e058ad4f4d342a274ba5f34fc6f17ef8e9ad8b91cdb
Successfully built deafrica-waterbodies
  Attempting uninstall: deafrica-waterbodies
    Found existing installation: deafrica-waterbodies 0.0.1
    Uninstalling deafrica-waterbodies-0.0.1:
      Successfully uninstalled deafrica-waterbodies-0.0.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is avail

## Load the necessary python packages

In [2]:
import os
import math
import subprocess
import geopandas as gpd

## Analysis Parameters

For each of the specified parameters below, see the [Turn water observations into waterbody polygons notebook](TurnWaterObservationsIntoWaterbodyPolygons.ipynb) for more details.

If you wish to filter out major rivers and CBD areas from your waterbodies, please add the flags, 
`--remove-major-rivers` and `--remove-cbd` respectively and set the paths to the vector datasets you would like to use for filtering using `--major-rivers-mask-fp='<path to dataset>` and `--urban-mask-fp=<path to dataset>`.

In [3]:
# Delete the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables to be able to use the 
# credentials in the ~/.aws/credentials file.
aws_default_config = {
    'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',}

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [4]:
# File path to the vector file defining our area of interest. 
run_type = "custom"
vector_file_fp = "/home/jovyan/Data/Waterbodies/InputDatasets/SenegalBasin.geojson"
primary_threshold = 0.1
secondary_threshold = 0.05
min_polygon_size = 4500
max_polygon_size = math.inf
land_sea_mask_fp = "/home/jovyan/Data/Waterbodies/InputDatasets/goas_v01.gpkg"
handle_large_polygons="erode-dilate-v2"
pp_test_threshold=0.005
product_version = "0.0.1"
output_bucket_name = "deafrica-waterbodies-dev"
output_filename="senegalbasinwaterbodies"
output_file_type= "Shapefile"
#output_file_type= "GeoJSON" 

In [5]:
#The script deafrica-waterbodies is installed in '/home/jovyan/.local/bin' which is not on PATH.
executable = "/home/jovyan/.local/bin/deafrica-waterbodies"
cmd = f"{executable} generate-waterbodies --run-type={run_type} --vector-file-fp={vector_file_fp} " \
f"--primary-threshold={primary_threshold} --secondary-threshold={secondary_threshold} " \
f"--min-polygon-size={min_polygon_size} --max-polygon-size={max_polygon_size} " \
f"--remove-ocean-polygons --land-sea-mask-fp={land_sea_mask_fp} --handle-large-polygons={handle_large_polygons} " \
f"--pp-test-threshold={pp_test_threshold} -v --product-version={product_version} --s3 --output-bucket-name={output_bucket_name} --output-file-name={output_filename} " \
f"--output-file-type={output_file_type}" 

cmd_list = cmd.split(" ")
cmd_list

['/home/jovyan/.local/bin/deafrica-waterbodies',
 'generate-waterbodies',
 '--run-type=custom',
 '--vector-file-fp=/home/jovyan/Data/Waterbodies/InputDatasets/SenegalBasin.geojson',
 '--primary-threshold=0.1',
 '--secondary-threshold=0.05',
 '--min-polygon-size=4500',
 '--max-polygon-size=inf',
 '--remove-ocean-polygons',
 '--land-sea-mask-fp=/home/jovyan/Data/Waterbodies/InputDatasets/goas_v01.gpkg',
 '--handle-large-polygons=erode-dilate-v2',
 '--pp-test-threshold=0.005',
 '-v',
 '--product-version=0.0.1',
 '--s3',
 '--output-bucket-name=deafrica-waterbodies-dev',
 '--output-file-name=senegalbasinwaterbodies',
 '--output-file-type=Shapefile']

In [6]:
subprocess.run(cmd_list)

Usage: deafrica-waterbodies [OPTIONS] COMMAND [ARGS]...
Try 'deafrica-waterbodies --help' for help.

Error: No such command 'generate-waterbodies'.


CompletedProcess(args=['/home/jovyan/.local/bin/deafrica-waterbodies', 'generate-waterbodies', '--run-type=custom', '--vector-file-fp=/home/jovyan/Data/Waterbodies/InputDatasets/SenegalBasin.geojson', '--primary-threshold=0.1', '--secondary-threshold=0.05', '--min-polygon-size=4500', '--max-polygon-size=inf', '--remove-ocean-polygons', '--land-sea-mask-fp=/home/jovyan/Data/Waterbodies/InputDatasets/goas_v01.gpkg', '--handle-large-polygons=erode-dilate-v2', '--pp-test-threshold=0.005', '-v', '--product-version=0.0.1', '--s3', '--output-bucket-name=deafrica-waterbodies-dev', '--output-file-name=senegalbasinwaterbodies', '--output-file-type=Shapefile'], returncode=2)

In [ ]:
# Read the output file.
output_fp = "s3://deafrica-waterbodies-dev/0-0-1/shapefile/senegalbasinwaterbodies.shp"
waterbodies =  gpd.read_file(output_fp)
waterbodies.head()

In [ ]:
# Plot the waterbodies. 
waterbodies.plot(edgecolor='k')